<h1>Estimating The Price of Hydrogen From Different Production Methods</h1>
<h3>Graham Smith, Feb 2020</h3>

<h3>Introduction</h3>
<p>This tool is designed to extend reports such as the International Energy Agency's <a href="http://example.com/">
G20 Hydrogen Report</a> by providing an interactive graphs that let users explore the impact of assumptions on hydrogen price. This is important as rapid changes in the price of renewable electricity,  electrolysers and gas means that reports based on data that are only a few years old can be misleading. The base case presented here is most relevant to the United Kingdom with the output in GBP but the tool should be globally applicable.  <em>The code preceedes the interactive graph, so you can skip to the bottom if you're not interested in checking the method or code.</em></p>

<h3>Setting Up Model</h3>

In [57]:
#Imports
import math
import bokeh
import bokeh.plotting as plt
from bokeh.models import Span, Label
from bokeh.io import show, push_notebook, output_notebook
output_notebook()

from ipywidgets import interact

Loading BokehJS ...

In [28]:
#Global Physical & Financial Variables
LHV = 33 #kWe kgH2 (Lower Heating Value - LHV)
discountRate = 8 #% - This is the effective annual cost of capital investment

<h3>PEM Electrolyser Data</h3>

In [54]:
#Usage
hoursPerDay = 24 #hours
daysPerYear = 340 #days
averageNominalPower = 80 #%
use = hoursPerDay * daysPerYear * averageNominalPower/100 #effective hours per year

#Electrolyser
efficiency = 69 #% equivalent to 1.78 V
efficiencyLoss = 1.5 #%efficiency loss per year equivalent to 2.25 muV operation-1 
lifeHours = 80000 #h of useable hours
lifeYears = lifeHours/(hoursPerDay * daysPerYear) #number of years electrolyser is in operation
powerRequired = LHV/(efficiency/100)# kW H2

#CAPEX
equipmentCost = 250 #GBP kW 250-1000 euro
CAPEX = equipmentCost*powerRequired/lifeHours #price per Kg H2 produced overlifetime

#FINANCING CAPEX
FINANCING = 0

#ENERGY OPEX
elecPrice = 0.03 #mean electricity price in GBP / kWh
energyOPEX = powerRequired*elecPrice #GBP per kg H2

#MAINTAINANCE OPEX
maintanceCost = 1.5 #1.5 % of CAPEX per year
maintOPEX = maintanceCost/100*equipmentCost*lifeYears/lifeHours

OPEX = maintOPEX + energyOPEX

total = FINANCING + CAPEX + OPEX #GBP kW-1

In [92]:
technology = ['PEME', 'Alkaline', 'SMR', 'ATR']
costs = ['Electricity', 'Gas', 'Other OpEx', 'CapEx', 'Finance']
colors = ['#005596', '#00AEEF', '#791D7E', '#6DB33F',  '#EE3224', '#FFC425'][0:len(costs)]

data = {'technology' : technology,
        'Electricity'   : [2, 1, 4, 3],
        'Gas'   : [5, 3, 4, 2],
        'Other OpEx'   : [3, 2, 4, 4],
        'CapEx'   : [3, 2, 4, 4],
        'Finance'   : [3, 2, 4, 4],       
       }

In [125]:
def update(a,b,c,d,e):
    data = {'technology' : technology,
        'Electricity'   : [a, 1, 4, 3],
        'Gas'   : [b, 3, 4, 2],
        'Other OpEx'   : [c, 2, 4, 4],
        'CapEx'   : [d, 2, 4, 4],
        'Finance'   : [e, 2, 4, 4],       
       }
    for i in range(5):
        vbarplot[i].data_source.data=data
    push_notebook()

In [126]:
height=int(500)
width = int(height*1.618)
p = plt.figure(x_range=technology, plot_height=height, plot_width=width,
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

vbarplot = p.vbar_stack(costs, x='technology', width=0.8, source=data, color=colors, legend_label=costs)


pumpPrice = Span(location=2, dimension='width', line_color='green', line_dash='dashed', line_width=3)
pumpPriceLabel = Label(x=4.5, y=1, text='Pump Price', render_mode='css', border_line_color=None)

p.add_layout(pumpPrice)
p.add_layout(pumpPriceLabel)


p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = 'Black'

p.y_range.end = 25
p.y_range.start = 0
p.legend.border_line_color = None
p.legend.location = 'top_center'
p.legend.orientation = 'horizontal'

t = show(p, notebook_handle=t)

In [129]:
interact(update, a=(0,10), b=(0,10), c=(0,10), d=(0,10), e=(0,10))

interactive(children=(IntSlider(value=5, description='a', max=10), IntSlider(value=5, description='b', max=10)…

<function __main__.update(a, b, c, d, e)>

<h3>Future Additions to Model</h3>
<ul>
    <li>compression costs for 7 MPa distribution network.</li>
    <li>distribution costs / local generation / central generation</li>
    <li>make a link to a polarisation curve and a price minimisation stragety</li>
    <li>can we make wind baseload like with hydrogen production</li>
    <li>how much hydrogen would we make compared to energy use?</li>
</ul>